# Understanding CPR policy violations 

CPR generation policy violations could happen due to a variety of reasons. It's hard to get a machine to predict these when we don't really know what's happening.

I'll start breaking the violations (according to the majority votes of our annotators) into whether each rule was violated. If this seems to show patterns, I'll ask one of the policy team to finish this off.

In [5]:
from pathlib import Path

import pandas as pd
import argilla as rg
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [6]:
feedback_from_hf = rg.FeedbackDataset.from_huggingface(
    "ClimatePolicyRadar/annotation-responses-unece"
)

argilla.yaml:   0%|          | 0.00/3.85k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1700 [00:00<?, ? examples/s]

Parsing records: 100%|██████████| 1700/1700 [00:31<00:00, 54.22it/s]


In [18]:
dataset_df = feedback_from_hf.format_as("datasets").to_pandas()
dataset_df["gen_uuid"] = dataset_df["external_id"].apply(lambda id: id.split("__")[0])
dataset_df.head(1)

,question,output,sources,overall-quality,overall-quality-suggestion,overall-quality-suggestion-metadata,system-responds,system-responds-suggestion,system-responds-suggestion-metadata,should-have-responded,...,cpr-generation-policy-suggestion-metadata,cpr-formatting-guidelines,cpr-formatting-guidelines-suggestion,cpr-formatting-guidelines-suggestion-metadata,comments,comments-suggestion,comments-suggestion-metadata,external_id,metadata,gen_uuid
0,Who owns the carbon certificates generated by ...,SERNANP owns the rights related to the ecosyst...,**[0]**\n Carbon credit certificates.- These a...,[],None,"{'type': None, 'score': None, 'agent': None}",[],None,"{'type': None, 'score': None, 'agent': None}",[],...,"{'type': None, 'score': None, 'agent': None}",[],None,"{'type': None, 'score': None, 'agent': None}",[],None,"{'type': None, 'score': None, 'agent': None}",00134b98bd7339ea01d58eca9383fad9___dd5e4e5d0e6...,"{""document_id"": ""CCLW.executive.1534.1800"", ""g...",00134b98bd7339ea01d58eca9383fad9


In [4]:
df = pd.read_json("../data/cpr_generation_policy_human_annotation_scores.jsonl", lines=True)
df.head()

,score,type,name,gen_uuid
0,1,cpr-generation-policy,human,00180743a629331588c7f92587375435
1,1,cpr-generation-policy,human,008c9d0c0346920c38ff1fed91c685ff
2,1,cpr-generation-policy,human,00b717a2102d0a4be92452f0d391889d
3,1,cpr-generation-policy,human,00cc9b801035ddcefd17c12e785b6874
4,1,cpr-generation-policy,human,011f57f2e02587ea7e231bb55199c2f9


In [23]:
generation_policy_scores_df = df.merge(dataset_df, left_on="gen_uuid", right_on="gen_uuid", how="left")[["gen_uuid", "question", "output", "sources", "score"]]
generation_policy_scores_df.drop_duplicates(subset=["gen_uuid"], inplace=True)
generation_policy_scores_df.head(1)



,gen_uuid,question,output,sources,score
0,00180743a629331588c7f92587375435,What are the specific objectives related to re...,- Step up research and development of technolo...,**[0]**\n - To step up research and developmen...,1


In [24]:
generation_policy_score_violated = generation_policy_scores_df[generation_policy_scores_df["score"] == 0]

generation_policy_score_violated.to_csv("../data/understanding_cpr_generation_policy_violations_human_annotation.csv", index=False)